In [1]:
%matplotlib inline

In [2]:
import pandas as pd
from psyrun.store.npz import NpzStore

In [3]:
data = NpzStore().load('../data/memory.npz')
data['dimensions'][np.logical_not(np.isfinite(data['dimensions']))] = -1
data['n_neurons'][np.logical_not(np.isfinite(data['n_neurons']))] = -1

In [4]:
import xarray as xr

In [5]:
data.keys()

dict_keys(['dimensions', 'time', 'memory', 'trial', 'event_times', 'n_neurons', 'model', 'backend'])

In [6]:
np.unique(data['model'])

array(['circ_conv', 'nback'], 
      dtype='<U9')

In [7]:
for x in zip(data['backend'], data['model'], data['trial']):
    if x[0] == 'reference' and x[1] == 'circ_conv':
        print(x)

('reference', 'circ_conv', 0)
('reference', 'circ_conv', 2)
('reference', 'circ_conv', 1)
('reference', 'circ_conv', 4)
('reference', 'circ_conv', 3)


In [8]:
ds = xr.Dataset({
    'memory': (['idx', 't_idx'], data['memory']),
    'time': (['idx', 't_idx'], data['time']),
    'trial': (['idx'], data['trial']),
    'n_neurons': (['idx'], data['n_neurons']),
    'dimensions': (['idx'], data['dimensions']),
    'backend': (['idx'], data['backend']),
    'model': (['model'], data['model']),
}, coords={'idx': range(len(data['trial'])), 't_idx': range(data['time'].shape[1])})

In [9]:
def get_coords(x):
    coords = np.unique(x)
    try:
        coords[np.logical_not(np.isfinite(coords))] = -1
    except:
        pass
    return np.unique(coords)

keys = ['model', 'backend', 'n_neurons', 'dimensions', 'trial']
coords = {k: get_coords(data[k]) for k in keys}
coords['t_idx'] = range(data['time'].shape[1])
shape = tuple(len(coords[k]) for k in keys) + (data['time'].shape[1],)

In [10]:
ds = xr.Dataset({
        'memory': (keys + ['t_idx'], np.empty(shape)),
        't': (keys + ['t_idx'], np.empty(shape)),
    }, coords=coords)
ds

<xarray.Dataset>
Dimensions:     (backend: 4, dimensions: 2, model: 2, n_neurons: 2, t_idx: 10075, trial: 5)
Coordinates:
  * dimensions  (dimensions) float64 -1.0 500.0
  * trial       (trial) int64 0 1 2 3 4
  * n_neurons   (n_neurons) float64 -1.0 500.0
  * model       (model) <U9 'circ_conv' 'nback'
  * backend     (backend) <U9 'ocl_cpu' 'ocl_gpu' 'optimized' 'reference'
  * t_idx       (t_idx) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
Data variables:
    t           (model, backend, n_neurons, dimensions, trial, t_idx) float64 0.0 ...
    memory      (model, backend, n_neurons, dimensions, trial, t_idx) float64 0.0 ...

In [11]:
import itertools
for x in itertools.product(*(coords[k] for k in keys)):
    sel = np.ones_like(data['memory'], dtype=bool)
    for i, k in enumerate(keys):
        sel = np.logical_and(sel, (data[k] == x[i])[:, None])
    loc = {k: x[i] for i, k in enumerate(keys)}
    if np.any(sel):
        ds['memory'].loc[loc] = data['memory'][sel]
        ds['t'].loc[loc] = data['time'][sel]

In [12]:
def mem_for(benchmark, backend, n_neurons=-1, dimensions=-1):
    da = ds.loc[{
        'model': benchmark,
        'backend': backend,
        'n_neurons': n_neurons,
        'dimensions': dimensions,
    }]
    return np.max(da['memory'] / 1024. / 1024.)

In [13]:
print('reference:', mem_for('circ_conv', 'reference', 500, 500))
print('optimized:', mem_for('circ_conv', 'optimized', 500, 500))

reference: <xarray.DataArray 'memory' ()>
array(291.359375)
Coordinates:
    dimensions  float64 500.0
    n_neurons   float64 500.0
    model       <U9 'circ_conv'
    backend     <U9 'reference'
optimized: <xarray.DataArray 'memory' ()>
array(414.796875)
Coordinates:
    dimensions  float64 500.0
    n_neurons   float64 500.0
    model       <U9 'circ_conv'
    backend     <U9 'optimized'


In [14]:
print('reference:', mem_for('nback', 'reference'))
print('optimized:', mem_for('nback', 'optimized'))

reference: <xarray.DataArray 'memory' ()>
array(184.87890625)
Coordinates:
    dimensions  float64 -1.0
    n_neurons   float64 -1.0
    model       <U9 'nback'
    backend     <U9 'reference'
optimized: <xarray.DataArray 'memory' ()>
array(184.890625)
Coordinates:
    dimensions  float64 -1.0
    n_neurons   float64 -1.0
    model       <U9 'nback'
    backend     <U9 'optimized'
